In [ ]:
import numpy as np
import cv2
import os


images_folder = './N2-embryo-10um-SRx5-crop-images-correct-angle-folders'

seg_folder = './N2-embryo-10um-SRx5-crop-seg-correct-angle-folders'


sub_folders = os.listdir(images_folder)

print(sub_folders)

pointcloud_folder = "N2-embryo-10um-SRx5-3D-pointcloud"


In [ ]:
import math
import open3d
from copy import deepcopy

fx_scale = 5.0 

for sub_folder in sub_folders:

    images_list_dir = images_folder+'/'+sub_folder
    seg_images_list_dir = seg_folder+'/'+sub_folder

    images_list = os.listdir(images_list_dir)
    seg_images_list = os.listdir(seg_images_list_dir)


    
    
    print(sub_folder)


    
    pcd_offset_file = open3d.io.read_point_cloud(pointcloud_folder+'/'+sub_folder+'.pcd')

    
    cl,index = pcd_offset_file.remove_statistical_outlier(nb_neighbors = 50,std_ratio= 1.0)
    origin_pcd = pcd_offset_file.select_by_index(index)

    surface_area = 0
    volume = 0

    length = 0
    max_width = 0

    for j in range(720):

        

        xyz_ppints = np.array(origin_pcd.points).astype(int)

        where_res=np.where(xyz_ppints[:,1] == j)

        xz_layer = xyz_ppints[where_res]

        xz_image = np.zeros((500,500))

        xz_data = []

        for point in xz_layer:
            x,y,z = point
            
            xz_data.append([x,z+250])

        if len(xz_data) >0:

            length += 1

            xz_data = np.array(xz_data).reshape(-1,1,2)

            approx1 = cv2.approxPolyDP(xz_data,3,False)

            res_hull=cv2.convexHull(approx1)

            xz_image  =cv2.polylines(xz_image,[res_hull],True,(255,255,255),1)


            
            rect = cv2.minAreaRect(res_hull)
            box = cv2.boxPoints(rect)
            box = np.int0(box)

            
            width = np.linalg.norm(box[0] - box[1])
            height = np.linalg.norm(box[1] - box[2])

            
            max_w_temp = max(width, height)
            

            if max_w_temp > max_width:
                max_width = max_w_temp

                cv2.drawContours(xz_image, [box], 0, (255, 255, 255), 1)
                cv2.imwrite('xz_layer.png',xz_image)


            
        

        


    print('length = ', length/fx_scale)
    print('max_width = ', max_width/fx_scale)
            